# Chapter 8.4 - Generating images with VAEs

In [1]:
import keras
from keras.layers import Input, Conv2D, Dense, Flatten
from keras import backend as K
from keras.models import Model
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_shape = (28, 28, 1)
batch_size = 16
# Dimensionality of the latent space: a plane
latent_dim = 2  

In [3]:
input_img = Input(shape = img_shape)

x = Conv2D(filters = 32, 
           kernel_size = (3, 3),
           padding = 'same', 
           activation = 'relu')(input_img)
x = Conv2D(filters = 64, 
           kernel_size = (3, 3),
           padding ='same', 
           activation = 'relu',
           strides = (2, 2))(x)
x = Conv2D(filters = 64, 
           kernel_size = (3, 3),
           padding = 'same', 
           activation = 'relu')(x)
x = Conv2D(filters = 64, 
           kernel_size = (3, 3),
           padding = 'same', 
           activation = 'relu')(x)
shape_before_flattening = K.int_shape(x)

x = Flatten()(x)
x = Dense(units = 32, 
          activation = 'relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

In [4]:
from keras.layers import Lambda, Reshape, Conv2DTranspose

In [5]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0], 
                                       latent_dim),
                              mean = 0., 
                              stddev = 1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

In [6]:
# This is the input where we will feed `z`.
decoder_input = Input(K.int_shape(z)[1:])

# Upsample to the correct number of units
x = Dense(units = np.prod(shape_before_flattening[1:]),
          activation='relu')(decoder_input)

# Reshape into an image of the same shape as before our last `Flatten` layer
x = Reshape(shape_before_flattening[1:])(x)

# We then apply then reverse operation to the initial
# stack of convolution layers: a `Conv2DTranspose` layers
# with corresponding parameters.
x = Conv2DTranspose(filters = 32, 
                    kernel_size = (3, 3),
                    padding='same', 
                    activation='relu',
                    strides = (2, 2))(x)
x = Conv2D(filters = 1, 
           kernel_size = (3, 3),
           padding = 'same', 
           activation = 'sigmoid')(x)

# We end up with a feature map of the same size as the original input.

# This is our decoder model.
decoder = Model(decoder_input, x)

# We then apply it to `z` to recover the decoded `z`.
z_decoded = decoder(z)